# HKL calculation, compared to SPEC results

In [ ]:
%matplotlib notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# from ophyd.hkl.diffract import E6C
from ophyd.hkl.calc import CalcE6C

## Load the desired HKL trajectory

In [ ]:
hkls = pd.read_csv('hkl_data/hkl.txt', delim_whitespace=True)
hkls.keys()

## Get the motor positions that SPEC calculated

In [ ]:
# The motor positions according to SPEC
spec_motors = pd.read_csv('hkl_data/motors.txt', delim_whitespace=True)
spec_motors.keys()

## Plot the trajectory of the physical motors

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12, 6),
                         subplot_kw={'xticks': []})
fig.subplots_adjust(hspace=0.3, wspace=0.2)

plt.suptitle('Trajectory according to SPEC')
for ax, key in zip(axes.flat, spec_motors.keys()):
    ax.plot(spec_motors.index, spec_motors[key], label=key)
    ax.set_title(key)
plt.show()

## Plot the desired HKL trajectory

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 6))
fig.subplots_adjust(hspace=0.4, wspace=0.2)

plt.suptitle('Desired HKL trajectory')
axes[0].plot(hkls.h)
axes[0].set_title('h')
axes[1].plot(hkls.k)
axes[1].set_title('k')
axes[2].plot(hkls.l)
axes[2].set_title('l')
plt.show()

## Initialize a calculation engine

In [ ]:
calc = CalcE6C(engine='hkl')
calc.wavelength = 1.33  # nm
print('physical axes', calc.physical_axes)
print('pseudo axes', calc.pseudo_axes)
print('omega parameter is', calc['omega'])

## Set some constraints on the physical motors

In [ ]:
phi = calc['phi']
phi.limits = (0, 0)
phi.value = 0
phi.fit = False

chi = calc['chi']
chi.limits = (-90, -90)
chi.value = -90
chi.fit = False

mu = calc['mu']
mu.limits = (0, 0)
mu.value = 0
mu.fit = False

print('phi', calc['phi'])
print('chi', calc['chi'])
print('mu', calc['mu'])

# Add a sample to work with

In [ ]:
# from ophyd.hkl.sample import HklSample
# new_sample supports kwargs (see `help(HklSample)`)
sample = calc.new_sample('sample0', lattice=(3.78, 3.78, 13.28, 90, 90, 90))

## Primary reflection

In [ ]:
# set the geometry of the real motor positions first
calc.physical_axis_values = calc.Position(mu=0.0, omega=71.04, chi=-90.0, phi=0.0, gamma=-1.65, delta=136.7)
# and then add the primary reflection
r1 = sample.add_reflection(0, 0, 2)

## Secondary reflection

In [ ]:
# set the geometry
calc.physical_axis_values = calc.Position(mu=0.0, omega=158.22, chi=-90.0, phi=0.0, gamma=1.7, delta=164.94)
# and the secondary reflection
r2 = sample.add_reflection(1, 0, 1)

## Calculate the UB matrix

In [ ]:
sample.compute_UB(r1, r2)
sample.UB

## Calculate the trajectory

In [ ]:
for seq, (h, k, l) in hkls.iterrows():
    print('-- hkl {} --'.format((h, k, l)))
    print('Solutions:')
    for sol in calc.calc((h, k, l)):
        print('\t{}'.format(sol))
        
    break